<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/scrp_unifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<ol>
<li>**CHECKED** Obtener nombres de las categorias a las que pertenecen los archivos</li>
<li>**CHECKED** Enviarlas a las carpetas a las que pertenecen con las fechas agregadas</li>
<li>verificar que todos los archivos tengan la columna de las fechas</li>
<li>Se crea una base maestra a partir de los registros más viejos</li>

<li>crear el ID concatenando ranking y url</li>
<li>si los 50 estan intactactos:</li>
        no se consideran
<li>si existe un cambio de posición</li>
        se agrega a la base maestra.
</ol>

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook

import os
from datetime import datetime
import time

import requests

import pandas as pd

import csv

In [16]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
br  logs  mx


In [17]:
entries_mx = os.listdir('mx')
print(entries_mx)
entries_br = os.listdir('br')
print(entries_br)

['grocery', 'automotive', 'amazon-devices', 'baby', 'sports', 'electronics', 'tools', 'kitchen', 'industrial', 'musical-instruments', 'toys', 'books', 'music', 'officeproduct', 'dvd', 'handmade', 'pet-supplies', 'shoes', 'hpc', 'software', 'digital-text', 'videogames', 'mx-test_grocery_2020-07-31 05_38.csv', 'mx-test_grocery_2020-07-31 05_38.xlsx', 'mx-test_automotive_2020-07-31 05_38.csv', 'mx-test_automotive_2020-07-31 05_38.xlsx', 'mx-test_amazon-devices_2020-07-31 05_38.csv', 'mx-test_amazon-devices_2020-07-31 05_38.xlsx', 'mx-test_baby_2020-07-31 05_38.csv', 'mx-test_baby_2020-07-31 05_38.xlsx', 'mx-test_sports_2020-07-31 05_38.csv', 'mx-test_electronics_2020-07-31 05_38.csv', 'mx-test_electronics_2020-07-31 05_38.xlsx', 'mx-test_tools_2020-07-31 05_38.csv', 'mx-test_tools_2020-07-31 05_38.xlsx', 'mx-test_kitchen_2020-07-31 05_38.csv', 'mx-test_kitchen_2020-07-31 05_38.xlsx', 'mx-test_industrial_2020-07-31 05_38.csv', 'mx-test_industrial_2020-07-31 05_38.xlsx', 'mx-test_musical-in

In [18]:
def rename_folders_files(str_dir, change_this, to_this):
    str_dir = str_dir + '/'
    for entry in str_dir:
        original_file = str_dir + entry

        entry_edited = entry.replace(change_this, to_this)
        edited_name = str_dir + entry_edited
        
        os.rename(original_file, edited_name)

In [19]:
date = datetime.now()
date

datetime.datetime(2020, 8, 5, 6, 13, 4, 987484)

In [20]:
def add_date(entry):
    '''Bbtain date and made it a Pandas Series'''
    date_file = entry[-20:]
    str_date = date_file[:16]
    date = datetime.strptime(str_date,'%Y-%m-%d %H_%M')
    date_array = [date]*50
    date_series = pd.Series(date_array, name='Date')

    category = entry[8:(len(entry)-21)]

    '''Concat the Date Serie to the main dataframe'''
    main_df = pd.read_csv(entry, sep='|')
    new_df = pd.concat([date_series, main_df], axis=1)
    new_df.to_csv(entry, sep='|', index= False)
    new_df

    file_name_len = (len(entry)-3)
    file_name = entry[:file_name_len]
    excel_name = file_name + 'xlsx'
    new_df.to_excel(excel_name, index= False)

In [21]:
countries= ['mx','br']

In [22]:
!ls

br  logs  mx


In [23]:
#por pais en paises
for country in countries:
    country_entries = os.listdir(country)
    
    #por carpeta en paises
    for entry in country_entries:
        try:
            
            #Archivos en carpetas
            folder_files = os.listdir(country + '/' + entry)
            if folder_files[:2] != country:
                csv_counter = 0
                eleven_cells = 0
                ten_cells = 0

                for file_content in folder_files:    
                    #Archivo csv
                    if file_content[-3:] == 'csv':
                        csv_counter = csv_counter + 1
                        complete_file = country + '/' + entry + '/' + file_content
                        with open(complete_file, mode="r", encoding='utf-8') as csv_file:
                            csv_reader = csv.reader(csv_file, delimiter='|')
                            header = next(csv_reader)
                            if len(header) == 10:
                                ten_cells = ten_cells + 1
                            elif len(header) == 11:
                                eleven_cells = eleven_cells + 1
                        csv_file.close()
                print(f'ten_s {ten_cells} | elevens {eleven_cells} | of {csv_counter} csv_files in {country} - {entry}')
        except:
            pass

ten_s 18 | elevens 42 | of 60 csv_files in mx - grocery
ten_s 17 | elevens 41 | of 58 csv_files in mx - automotive
ten_s 16 | elevens 36 | of 53 csv_files in mx - amazon-devices
ten_s 17 | elevens 40 | of 57 csv_files in mx - baby
ten_s 17 | elevens 35 | of 52 csv_files in mx - electronics
ten_s 16 | elevens 35 | of 51 csv_files in mx - tools
ten_s 16 | elevens 35 | of 51 csv_files in mx - kitchen
ten_s 16 | elevens 34 | of 50 csv_files in mx - industrial
ten_s 16 | elevens 34 | of 50 csv_files in mx - musical-instruments
ten_s 16 | elevens 34 | of 50 csv_files in mx - toys
ten_s 16 | elevens 33 | of 49 csv_files in mx - books
ten_s 16 | elevens 33 | of 49 csv_files in mx - music
ten_s 16 | elevens 33 | of 49 csv_files in mx - officeproduct
ten_s 16 | elevens 33 | of 49 csv_files in mx - dvd
ten_s 16 | elevens 33 | of 49 csv_files in mx - handmade
ten_s 16 | elevens 33 | of 49 csv_files in mx - pet-supplies
ten_s 16 | elevens 33 | of 49 csv_files in mx - shoes
ten_s 16 | elevens 33 | o